In [ ]:
!pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO
import numpy as np
import cv2
import random

In [ ]:
train_images = 'BoneFractureYolo8/train/images'
train_labels = 'BoneFractureYolo8/train/labels'

test_images = 'BoneFractureYolo8/test/images'
test_labels = 'BoneFractureYolo8/test/labels'

val_images = 'BoneFractureYolo8/valid/images'
val_labels = 'BoneFractureYolo8/valid/labels'

In [ ]:
print('Number of train frames: ' + str(len(os.listdir(train_images))))
print('Number of train labels: ' + str(len(os.listdir(train_labels))))
print('Number of val frames: ' + str(len(os.listdir(val_images))))
print('Number of val labels: ' + str(len(os.listdir(val_labels))))
print('Number of test frames: ' + str(len(os.listdir(test_images))))
print('Number of test labels: ' + str(len(os.listdir(test_labels))))
print('Total frames: ' + str(len(os.listdir(train_images)) + len(os.listdir(test_images)) + len(os.listdir(val_images))))

In [ ]:
image_files = [f for f in os.listdir(train_images) if os.path.isfile(os.path.join(train_labels, os.path.splitext(f)[0] + ".txt"))]
print(f"Number of valid images: {len(image_files)}")
valid_images = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    with open(label_path, "r") as f:
        labels = f.read().strip()
    if labels: 
        valid_images.append(image_file)

print(f"Number of valid images: {len(valid_images)}")
random_images = random.sample(valid_images, 16)
fig, axs = plt.subplots(4, 4, figsize=(16, 16))
for i, image_file in enumerate(random_images):
    row = i // 4
    col = i % 4
    image_path = os.path.join(train_images, image_file)
    image = cv2.imread(image_path)
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")
    for label in labels:
        label_parts = label.split()
        _, x1, y1, x2, y2, x3, y3, x4, y4, _ = map(float, label_parts)  # Usa i quattro vertici
        pts = [
            (int(x1 * image.shape[1]), int(y1 * image.shape[0])),
            (int(x2 * image.shape[1]), int(y2 * image.shape[0])),
            (int(x3 * image.shape[1]), int(y3 * image.shape[0])),
            (int(x4 * image.shape[1]), int(y4 * image.shape[0]))
        ]
        pts = np.array(pts, np.int32).reshape((-1, 1, 2))
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)
    axs[row, col].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[row, col].axis('off')

plt.show()

In [ ]:
def rotate_label(label_path, new_label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    new_lines = []
    for line in lines:
        parts = line.strip().split()
        parts = [parts[0]] + [1 - x for x in parts[1:]]
        new_lines.append(" ".join(map(str, parts)))
    with open(new_label_path, 'w') as file:
        file.writelines(new_lines)

def process_images_labels(image_folder, label_folder):
    for label_file in os.listdir(label_folder):
        if not label_file.endswith('.txt'):
            continue
        label_path = os.path.join(label_folder, label_file)
        image_path = os.path.join(image_folder, label_file.replace('.txt', '.jpg'))  
        new_image_path = os.path.join(image_folder, label_file.replace('.txt', '_rotated.jpg'))
        new_label_path = os.path.join(label_folder, label_file.replace('.txt', '_rotated.txt'))
        if os.path.exists(image_path):
            with open(label_path, 'r') as f:
                if f.read().strip(): 
                    image = cv2.imread(image_path)
                    rotated_image = cv2.rotate(image, cv2.ROTATE_180)
                    cv2.imwrite(new_image_path, rotated_image)
                    rotate_label(label_path, new_label_path)

In [ ]:
process_images_labels(train_images, train_labels)

In [ ]:
model=YOLO("yolov8m-obb.pt")

In [ ]:
result = model.train(data="BoneFractureYolo8/data.yaml",epochs=30, seed=46)

In [ ]:
df = pd.read_csv('runs/detect/train/results.csv')
df.columns = df.columns.str.strip()

# create subplots using seaborn
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

# plot the columns using seaborn
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])

# set titles and axis labels for each subplot
axs[0,0].set(title='Train Box Loss')
axs[0,1].set(title='Train Class Loss')
axs[1,0].set(title='Train DFL Loss')
axs[1,1].set(title='Metrics Precision (B)')
axs[2,0].set(title='Metrics Recall (B)')
axs[2,1].set(title='Metrics mAP50 (B)')
axs[3,0].set(title='Metrics mAP50-95 (B)')
axs[3,1].set(title='Validation Box Loss')
axs[4,0].set(title='Validation Class Loss')
axs[4,1].set(title='Validation DFL Loss')

# add suptitle and subheader
plt.suptitle('Training Metrics and Loss', fontsize=24)

# adjust top margin to make space for suptitle
plt.subplots_adjust(top=0.8)

# adjust spacing between subplots
plt.tight_layout()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
model = YOLO("/kaggle/working/runs/obb/train/weights/best.pt")
source = "/kaggle/input/bone-fracture-detection-computer-vision-project/BoneFractureYolo8/valid/images/image1_1773_png.rf.51512561cfc16438d9c13166f1b5457b.jpg"
results = model(source, box=0)  # Run inference with standard bounding boxes
for result in results:
    result.show()  # Show annotated image

    # Extract OBB information (if available)
    if result.obb is not None:
        obb_results = result.obb.xyxyxyxy  # OBB coordinates in xywh format
        print(obb_results)